# Convert .jpg into .rgb image

In [13]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install torchvision

Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [23]:
from PIL import Image
from pathlib import Path
import numpy as np
import tensorflow as tf
import glob
import os
import random

## Object detection

In [24]:
def preprocess_image_quantized(img_path, output_path, resolution, scale, zero_point, dtype):
    """
    Matches PyTorch ToTensor() preprocessing:
    - Resize to (resolution,resolution)
    - Convert to RGB
    - Scale to [0,1] float
    - Quantize to int8/uint8 with given scale & zero_point
    - Save as raw .rgb file in NHWC
    """
    # Load and resize
    image = Image.open(img_path).convert('RGB')
    image = image.resize((resolution, resolution), Image.BILINEAR)

    # ToTensor() behavior: float32 in [0,1]
    x = np.array(image, dtype=np.float32) / 255.0   # [H,W,3]

    # Quantize
    if dtype.lower() == "uint8":
        q = np.round(x / scale + zero_point).clip(0, 255).astype(np.uint8)
    elif dtype.lower() == "int8":
        q = np.round(x / scale + zero_point).clip(-128, 127).astype(np.int8)
    else:
        raise ValueError("dtype must be 'uint8' or 'int8'")

    # Add batch dim: [1,H,W,3]
    q = np.expand_dims(q, axis=0)

    # Save
    q.tofile(output_path)
    print(f"[OK] Saved quantized NHWC to {output_path} shape={q.shape} dtype={q.dtype} scale={scale} zp={zero_point}")

In [25]:
def reconstruct_quantized_rgb(rgb_bin_path, out_jpeg_path, resolution, scale, zero_point, dtype):
    # Determine numpy dtype from quantized dtype string
    np_dtype = np.uint8 if dtype.lower() == "uint8" else np.int8
    print(f"[INFO] Reading as {np_dtype}")

    # Read binary and reshape to NHWC
    q = np.fromfile(rgb_bin_path, dtype=np_dtype).reshape(1, resolution, resolution, 3)

    # Dequantize to float [0,255]
    x = (q.astype(np.float32) - float(zero_point)) * float(scale) * 255.0  # match ToTensor() inverse

    # Clip to valid range and convert to uint8
    x = np.clip(x, 0, 255).astype(np.uint8)

    # Convert to PIL Image (drop batch dim)
    img = Image.fromarray(x[0])
    img.save(out_jpeg_path, format="JPEG")
    print(f"[OK] Reconstructed preview to {out_jpeg_path}, shape={x[0].shape}, dtype=uint8")

In [26]:
model_path   = "./yolo_mcunet_model.tflite"
src_jpeg_dir = "./images"
out_rgb_dir  = "./rgb_images"
out_preview_dir = "./preview_images"

In [27]:
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()[0]
input_shape = input_details["shape"]  # [1, H, W, 3]
H, W = int(input_shape[1]), int(input_shape[2])
if input_shape[-1] != 3:
    raise ValueError(f"Expected 3 channels, got {input_shape[-1]}")
input_scale, input_zp = input_details["quantization"]
dtype_np = input_details["dtype"]
if dtype_np == np.int8:
    dtype_str = "int8"
elif dtype_np == np.uint8:
    dtype_str = "uint8"
else:
    raise ValueError(f"Unsupported dtype: {dtype_np}")

print(f"Model input: {H}x{W}x3  dtype={dtype_str}  scale={input_scale}  zp={input_zp}")

Model input: 160x160x3  dtype=int8  scale=0.003921568859368563  zp=-128


In [28]:
jpgs = sorted(glob.glob(os.path.join(src_jpeg_dir, "*.jpg")))
if not jpgs:
    raise FileNotFoundError(f"No .jpg files found under {src_jpeg_dir}")
random.seed(42)
sample = random.sample(jpgs, k=100)
print(f"Selected {len(sample)} images from {len(jpgs)} available.")

Selected 100 images from 1981 available.


In [29]:
index = []
for idx, jpg in enumerate(sample, 1):
    stem = Path(jpg).stem
    out_rgb_path = os.path.join(out_rgb_dir, f"{stem}_q_{H}.rgb")
    preprocess_image_quantized(
        jpg, out_rgb_path, H, input_scale, input_zp, dtype_str
    )

    preview_path = None
    if out_preview_dir:
        preview_path = os.path.join(out_preview_dir, f"{stem}_preview.png")
        reconstruct_quantized_rgb(out_rgb_path, preview_path, H, input_scale, input_zp, dtype_str)

[OK] Saved quantized NHWC to ./rgb_images/2012_002893_q_160.rgb shape=(1, 160, 160, 3) dtype=int8 scale=0.003921568859368563 zp=-128
[INFO] Reading as <class 'numpy.int8'>
[OK] Reconstructed preview to ./preview_images/2012_002893_preview.png, shape=(160, 160, 3), dtype=uint8
[OK] Saved quantized NHWC to ./rgb_images/2012_000567_q_160.rgb shape=(1, 160, 160, 3) dtype=int8 scale=0.003921568859368563 zp=-128
[INFO] Reading as <class 'numpy.int8'>
[OK] Reconstructed preview to ./preview_images/2012_000567_preview.png, shape=(160, 160, 3), dtype=uint8
[OK] Saved quantized NHWC to ./rgb_images/2012_000139_q_160.rgb shape=(1, 160, 160, 3) dtype=int8 scale=0.003921568859368563 zp=-128
[INFO] Reading as <class 'numpy.int8'>
[OK] Reconstructed preview to ./preview_images/2012_000139_preview.png, shape=(160, 160, 3), dtype=uint8
[OK] Saved quantized NHWC to ./rgb_images/2012_003351_q_160.rgb shape=(1, 160, 160, 3) dtype=int8 scale=0.003921568859368563 zp=-128
[INFO] Reading as <class 'numpy.int8